In [1]:
# Polynomial Module 
from operator import add
from operator import neg
from operator import mod
from fractions import Fraction as frac
from numpy.polynomial import polynomial as P

# Resize Adds Leading Zeros to the polynomial vectors

#Helper Functions 
# Extended Euclidean Algo for Integers

def egcd(a, b):
    x,y, u,v = 0,1, 1,0
    while a != 0:
        q, r = b//a, b % a
        m, n = x-u * q, y-v * q
        b,a, x,y, u,v = a,r, u,v, m,n
    gcdVal = b
    return gcdVal, x, y


#Modular inverse
#An application of extended GCD algorithm to finding modular inverses:
def modinv(a, m):
    gcdVal, x, y = egcd(a, m)
    if gcdVal != 1:
        return None  # modular inverse does not exist
    else:
        return x % m

#Modulus Function which handles Fractions aswell
def fracMod(f,m):
	[tmp,t1,t2]=egcd(f.denominator,m)
	if tmp!=1:
		print("ERROR GCD of denominator and m is not 1")
		return 0
	else:
		out=modinv(f.denominator,m)*f.numerator % m
		return out


def resize(c1,c2):
	if(len(c1)>len(c2)):
		c2=c2+[0]*(len(c1)-len(c2))
	if(len(c1)<len(c2)):
		c1=c1+[0]*(len(c2)-len(c1))
	return [c1,c2]

# Removes Leading Zeros
def trim(seq):
	if len(seq) == 0:
		return seq
	else:
		for i in range(len(seq) - 1, -1, -1):
			if seq[i] != 0:
				break
		return seq[0:i+1]

# Subtracts two Polnomials
def subPoly(c1,c2):
	[c1,c2]=resize(c1,c2)	
	c2=list(map(neg,c2))
	out=list(map(add, c1, c2))
	return trim(out)

# Adds two Polynomials
def addPoly(c1,c2):
	[c1,c2]=resize(c1,c2)
	out=list(map(add, c1, c2))
	return trim(out)

# Multiply two Polynomials 
def multPoly(c1,c2):
        order=(len(c1)-1+len(c2)-1)
        out=[0]*(order+1)
        for i in range(0,len(c1)):
                for j in range(0,len(c2)):
                        out[j+i]=out[j+i]+c1[i]*c2[j]
        return trim(out)

#Divides two polynomials
def divPoly(N,D):
	N, D = list(map(frac,trim(N))), list(map(frac,trim(D)))
	degN, degD = len(N)-1, len(D)-1
	if(degN>=degD):
		q=[0]*(degN-degD+1)
		while(degN>=degD and N!=[0]):
			d=list(D)
			[d.insert(0,frac(0,1)) for i in range(degN-degD)]
			q[degN-degD]=N[degN]/d[len(d)-1]
			d=list(map(lambda x: x*q[degN-degD],d))
			N=subPoly(N,d)
			degN=len(N)-1
		r=N	
	else:
		q=[0]
		r=N
	return [trim(q),trim(r)]

# Polynomial Coefficients mod k
def modPoly(c,k):
	if(k==0):
		print("Error in modPoly(c,k). Integer k must be non-zero")
	else:
		return list(map(lambda x: fracMod(x,k),c))

# Centerlift of Polynomial with respect to q
def cenPoly(c,q):
	u=float(q)/float(2)
	l=-u
	c=modPoly(c,q)
	c=list(map(lambda x: mod(x,-q) if x>u else x,c))
	c=list(map(lambda x: mod(x,q) if x<=l else x,c))
	return c

# Extended Euclidean Algorithm
def extEuclidPoly(a,b):
    switch = False
    a=trim(a)
    b=trim(b)
    if len(a)>=len(b):
        a1, b1 = a, b
    else:
        a1, b1 = b, a
        switch = True
    Q,R=[],[]
    while b1 != [0]:
        [q,r]=divPoly(a1,b1)
        Q.append(q)
        R.append(r)
        a1=b1
        b1=r
    S=[0]*(len(Q)+2)
    T=[0]*(len(Q)+2)
    S[0],S[1],T[0],T[1] = [1],[0],[0],[1]
    for x in range(2, len(S)):
        S[x]=subPoly(S[x-2],multPoly(Q[x-2],S[x-1]))
        T[x]=subPoly(T[x-2],multPoly(Q[x-2],T[x-1]))

    gcdVal=R[-2]
    s_out=S[-2]
    t_out=T[-2]
    ### ADDITIONAL STEPS TO SCALE GCD SUCH THAT LEADING TERM AS COEF OF 1:
    scaleFactor=gcdVal[len(gcdVal)-1]
    gcdVal=list(map(lambda x:x/scaleFactor,gcdVal))
    s_out=list(map(lambda x:x/scaleFactor,s_out))
    t_out=list(map(lambda x:x/scaleFactor,t_out))
    if switch:
        return [gcdVal,t_out,s_out]
    else:
        return [gcdVal,s_out,t_out]
	


def isTernary(f,alpha,beta):
	ones=0
	negones=0
	for i in range(0,len(f)):
		if(f[i]==1):
			ones=ones+1
		if(f[i]==-1):
			negones=negones+1
	if (negones+ones)<=len(f) and alpha==ones and beta==negones :
		return True
	else:
		return False


In [2]:
# Implementation of NTRU encryption and decryption
import math
from fractions import gcd
import poly

class Ntru:
	N, p, q, d = None, None, None, None
	f, g, h = None, None, None
	f_p, f_q, D = None, None, None

	def __init__(self, N_new, p_new, q_new):
		self.N = N_new
		self.p = p_new
		self.q = q_new
		D = [0] * (self.N + 1)
		D[0] = -1
		D[self.N] = 1
		self.D = D
		
	def genPublicKey(self, f_new, g_new, d_new):
		# Using Extended Euclidean Algorithm for Polynomials to get s and t. 
		self.f = f_new
		self.g = g_new
		self.d = d_new
		[gcd_f, s_f, t_f] = poly.extEuclidPoly(self.f, self.D)
		self.f_p = poly.modPoly(s_f, self.p)
		self.f_q = poly.modPoly(s_f, self.q)
		self.h = self.reModulo(poly.multPoly(self.f_q, self.g), self.D, self.q)
		if not self.runTests():
			quit()
			
	def getPublicKey(self):
		return self.h

	def setPublicKey(self,public_key):
		self.h = public_key
	
	def encrypt(self,message,randPol):
		if self.h != None:
			e_tilda = poly.addPoly(poly.multPoly(poly.multPoly([self.p], randPol), self.h), message)
			e = self.reModulo(e_tilda, self.D, self.q)
			return e
		else:
			print("Error: Public Key is not set, ", end = '')
			print("no default value of Public Key is available.")

	def decryptSQ(self,encryptedMessage):
		F_p_sq = poly.multPoly(self.f_p, self.f_p)
		f_sq = poly.multPoly(self.f, self.f)
		temp = self.reModulo(poly.multPoly(f_sq, encryptedMessage), self.D, self.q)
		centered = poly.cenPoly(temp, self.q)
		m1 = poly.multPoly(F_p_sq, centered)
		temp = self.reModulo(m1, self.D, self.p)
		return poly.trim(temp) 

	def decrypt(self,encryptedMessage):
		temp = self.reModulo(poly.multPoly(self.f, encryptedMessage), self.D, self.q)
		centered = poly.cenPoly(temp, self.q)
		m1 = poly.multPoly(self.f_p, centered)
		temp = self.reModulo(m1, self.D, self.p)
		return poly.trim(temp)

	def reModulo(self, num ,div, modby):
		[_,remain] = poly.divPoly(num, div)
		return poly.modPoly(remain, modby)
	
	def isPrime(self):
	    if self.N % 2 == 0 and self.N > 2: 
	        return False
	    return all(self.N % i for i in range(3, int(math.sqrt(self.N)) + 1, 2))
		
	def runTests(self):			
		# Checking if inputs satisfy conditions
		if not self.isPrime() :
			print("Error: N is not prime!")
			return False
	
		if gcd(self.N, self.p) != 1 :
			print("Error: gcd(N, p) is not 1")
			return False

		if gcd(self.N, self.q)!=1 :
			print("Error: gcd(N,q) is not 1")
			return False
	
		if self.q <= (6 * self.d + 1) * self.p:
			print("Error: q <= (6*d+1)*p")
			return False
	
		if not poly.isTernary(self.f, self.d+1, self.d):
			print("Error: f does not belong to T(d+1, d)")
			return False
		
		if not poly.isTernary(self.g, self.d, self.d):
			print("Error: g does not belong to T(d, d)")
			return False
        
		return(True)


In [3]:
from ntru import Ntru

#Bob
print("Public Key is generated by using Parameters: ", end = '')
print("N = 7, p = 29, and q = 491531")
Bob = Ntru(7, 29, 491531)
# f(x) = 1 + x - x^2 - x^4 + x^5
f = [1, 1, -1, 0, -1, 1]
# g(x) = -1 + x^2 + x^3 - x^6
g = [-1, 0, 1, 1, 0, 0, -1]
d = 2
print("f(x)= ", f)
print("g(x)= ", g)
print("d   = ", d, "\n\n")

Bob.genPublicKey(f, g, 2)
pub_key = Bob.getPublicKey()
print("Public Key Generated by Bob: ", pub_key, "\n\n")

#Alice
Alice = Ntru(7, 29, 491531)
Alice.setPublicKey(pub_key)
message = [1, 0, 1, 0, 1, 1, 1]
print("Original Message to be send: ", message)

# r(x) = -1 - x + x^2 + x^3
ranPol = [-1, -1, 1, 1]
print("Random Polynomial: ", ranPol)
encryptedMessage = Alice.encrypt(message, ranPol)
print("Encrypted Message to be send: ", encryptedMessage, "\n\n")

#BOB
print("Bob decrypts message sent to him")
print("Decrypted Message: ", Bob.decrypt(encryptedMessage))


Public Key is generated by using Parameters: N = 7, p = 29, and q = 491531
f(x)=  [1, 1, -1, 0, -1, 1]
g(x)=  [-1, 0, 1, 1, 0, 0, -1]
d   =  2 


Public Key Generated by Bob:  [394609, 27692, 62307, 263073, 346149, 41538, 339225] 


Original Message to be send:  [1, 0, 1, 0, 1, 1, 1]
Random Polynomial:  [-1, -1, 1, 1]
Encrypted Message to be send:  [283889, 269991, 484569, 353054, 179995, 159222, 235409] 


Bob decrypts message sent to him
Decrypted Message:  [1, 0, 1, 0, 1, 1, 1]
